# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [1]:
import glob
import os
import btrack
print('Btrack version', btrack.__version__)
import napari
print('Napari version', napari.__version__)
import numpy as np

from skimage.io import imread
from natsort import natsorted
from napari_animation import AnimationWidget
from tqdm import tqdm
import tifffile as tiff

Btrack version 0.4.1
Napari version 0.4.7


# Load experiment and position list

In [2]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
print('Experiment/position list', [(expt,natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])) for expt in expt_IDs])

Experiment/position list [('MK0000', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']), ('MK0001', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15'])]


# Manually select one position 

In [3]:
expt = 'MK0000'
pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
pos = 'Pos15'

# Load masks

In [4]:
mask_path = os.path.join(root_folder, '{}/{}/{}_cp_masks'.format(expt, pos, pos))
masks = natsorted(glob.glob(os.path.join(mask_path, '*.png')))
masks

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000001_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000002_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000003_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000004_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000005_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Mana

# Filter according to channel

In [5]:
gfp_masks = [file for file in masks if 'channel001' in file]
rfp_masks = [file for file in masks if 'channel002' in file]

# Load corresponding fluorescence images

In [6]:
image_path = os.path.join(root_folder, '{}/{}/{}_raw'.format(expt, pos, pos))
images = natsorted(glob.glob(os.path.join(image_path, '*.tif')))
gfp_images = [im for im in images if 'channel001' in im]
rfp_images = [im for im in images if 'channel002' in im]
irfp_images = [im for im in images if 'channel003' in im]

In [ ]:
irfp_images

# Create stack using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [14]:
def stack_generator(files):
    """Stack generator"""
    
    for filename in tqdm(files):
        img = imread(filename)
        yield img

In [15]:
gfp_mask_stack = stack_generator(gfp_masks)
rfp_mask_stack = stack_generator(rfp_masks)

gfp_image_stack = stack_generator(gfp_images)
rfp_image_stack = stack_generator(rfp_images)
irfp_image_stack = stack_generator(irfp_images)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

two channel option

In [ ]:
gfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, gfp_image_stack,
properties = ('area', 'mean_intensity')
)
rfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    rfp_mask_stack, rfp_image_stack,
    properties = ('area', 'mean_intensity')
)
irfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, irfp_image_stack,
properties = ('area', 'mean_intensity')
)

In [16]:
irfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, irfp_image_stack,
properties = ('area', 'mean_intensity')
)

[INFO][2021/05/30 04:21:32 pm] Localizing objects from segmentation...
[INFO][2021/05/30 04:21:32 pm] Found intensity_image data
[INFO][2021/05/30 04:21:32 pm] Calculating weighted centroids using intensity_image
 15%|█▍        | 149/1013 [00:26<02:19,  6.18it/s]


 30%|██▉       | 300/1013 [00:55<02:50,  4.17it/s]


 45%|████▍     | 451/1013 [01:27<02:02,  4.57it/s]


 59%|█████▉    | 602/1013 [02:04<01:43,  3.99it/s]


 74%|███████▍  | 753/1013 [02:45<01:27,  2.98it/s]


 89%|████████▉ | 904/1013 [03:28<00:27,  3.99it/s]


100%|██████████| 1013/1013 [03:59<00:00,  4.22it/s]
[INFO][2021/05/30 04:25:32 pm] Objects are of type: <class 'dict'>
[INFO][2021/05/30 04:25:37 pm] ...Found 551730 objects in 1013 frames.


In [ ]:
# inspect the first object
irfp_obj_from_generator[0]

## run btrack with the objects

We will use the objects from the generator here.

In [17]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    #tracker.append(gfp_obj_from_arr) #obj_from_generator)
    tracker.append(irfp_obj_from_generator)

    # set the volume
    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    #tracker.export(os.path.join(root_folder,expt, pos, 'irfp_tracks.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
#     #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
    
#     gfp_tracks = tracker.tracks

    irfp_data, irfp_properties, irfp_graph = tracker.to_napari(ndim=2)
    
    irfp_tracks = tracker.tracks

[INFO][2021/05/30 04:25:37 pm] Loaded btrack: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/05/30 04:25:37 pm] btrack (v0.4.1) library imported
[INFO][2021/05/30 04:25:37 pm] Setting max XYZ search radius to: 100
[INFO][2021/05/30 04:25:37 pm] Starting BayesianTracker session
[INFO][2021/05/30 04:25:37 pm] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json
[INFO][2021/05/30 04:25:37 pm] Loading motion model: b'MDCK_motion_Kristina'
[INFO][2021/05/30 04:25:37 pm] Setting max XYZ search radius to: 10
[INFO][2021/05/30 04:25:37 pm] Objects are of type: <class 'list'>
[INFO][2021/05/30 04:25:38 pm] Set volume to ((0, 1688), (0, 1352), (-100000.0, 100000.0))
[INFO][2021/05/30 04:25:38 pm] Starting tracking... 
[INFO][2021/05/30 04:25:38 pm] Tracking objects in frames 0 to 99 (of 1013)...
[INFO][2021/05/30 04:25:38 pm]  - Timing (Bayesian updates: 4.58ms, Linking: 0.46ms)
[INFO][202

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker usi#gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
    
#     gfp_tracks = tracker.tracksng a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_scribble_sparse.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    #tracker.append(rfp_obj_from_arr) #obj_from_generator)
    tracker.append(rfp_obj_from_generator)

    # set the volume
    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(root_folder, pos, 'rfp_tracks.h5'), obj_type='obj_type_2')

    # get the tracks in a format for napari visualization
    rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
    
    rfp_tracks = tracker.tracks

# load tracks from pre-config h5 files

In [7]:
tracks_path = natsorted(glob.glob('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/{}/{}/*.h5'.format(expt, pos)))
[(i, name) for i,name in enumerate(tracks_path)]

[(0,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/_irfp_tracks.h5'),
 (1,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/gfp_tracks_2021_05_21_05_57_38_PM.h5'),
 (2,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/gfp_tracks_2021_05_28_07_20_04_PM.h5'),
 (3,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5'),
 (4,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_21_05_57_45_PM.h5'),
 (5,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_28_07_20_11_PM.h5')]

In [8]:
gfp_tracks = tracks_path[2]
rfp_tracks = tracks_path[5]
irfp_tracks = tracks_path[3]

with btrack.dataio.HDF5FileHandler(irfp_tracks, 'r', obj_type = "obj_type_2") as hdf:
    irfp_tracks = hdf.tracks
# with btrack.dataio.HDF5FileHandler(gfp_tracks, 'r', obj_type = "obj_type_3") as hdf:
#     wt_tracks = hdf.tracks
# with btrack.dataio.HDF5FileHandler(rfp_tracks, 'r', obj_type = "obj_type_2") as hdf:
#     scr_tracks = hdf.tracks
# print("Tracks loaded")

[INFO][2021/05/30 04:16:36 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5...
[INFO][2021/05/30 04:16:36 PM] Loading tracks/obj_type_2
[INFO][2021/05/30 04:16:36 PM] Loading objects/obj_type_2 (551730, 5) (551730 filtered: None)
[INFO][2021/05/30 04:16:42 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5


# Inspect irfp tracks

In [18]:
irfp_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,area,mean_intensity
0,70,0,290.582676,103.409481,0.0,70,70,5,0,False,855.0,51.444444
1,70,1,290.582676,103.409481,0.0,70,70,5,0,True,NaN,NaN
2,70,2,285.010677,107.029442,0.0,70,70,5,0,False,827.0,61.153567
3,70,3,287.318953,104.421057,0.0,70,70,5,0,False,634.0,62.996845
4,70,4,286.586130,105.112755,0.0,70,70,5,0,False,776.0,64.275773


In [ ]:
### filter tracks the same way i filter the glimpses
focal_time_range = (0, 800) ### this particular expt stays in focus in this time range
long_tracks = [track for track in irfp_tracks if len(track) > 200]

### filter tracks that spend a lot of time out of focus
tracks = [track for track in long_tracks if track.t[-1] < focal_time_range[1]]

In [ ]:
tracks[0]

# add image layers

### compile stacks if necessary

In [ ]:
### compile stacks first if necessary 
root_folder = os.path.dirname(PATH)
pos = os.path.basename(root_folder)
### make stack folder if necessary
output_dir = os.path.join(root_folder,pos+'_stacks')
os.mkdir(output_dir) if os.path.exists(output_dir) == False else print(output_dir, 'already exists')
### count number of channels
raw_file_dir = os.path.join(root_folder, pos+'_raw')
file_list = natsorted(os.listdir(raw_file_dir))
channels = list(set([fn.split('channel')[1][0:3] for fn in file_list]))
for channel in ['000', '001', '002']:#channels:
    channel_file_list = natsorted([fn for fn in os.listdir(raw_file_dir) if 'channel'+channel in fn])
    stack = []
    for file in tqdm(channel_file_list):
        file = os.path.join(raw_file_dir, file)
        tif = np.array(tiff.imread(file),dtype=np.uint8)
        ### crop??
#         tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
#         tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
        stack.append(tif)
    stack = np.stack(stack,axis=0)
    with tiff.TiffWriter(os.path.join(output_dir,'channel{}.tif'.format(channel)),imagej=True) as output_file:
        output_file.save(stack)
    print('channel',channel, 'saved out')
    

In [19]:
#image_path = output_dir#'/home/nathan/data/kraken/pcna/MK0000/Pos15/stacks'
image_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/{}/{}/{}_stacks'.format(expt, pos, pos)
image_paths = natsorted(glob.glob(os.path.join(image_path, '*.tif')))
[(i, name) for i,name in enumerate(image_paths)]

[(0,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel000.tif'),
 (1,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel001.tif'),
 (2,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel002.tif'),
 (3,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel003.tif')]

In [20]:
irfp = imread(image_paths[3])
#rfp = imread(image_paths[2])

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [22]:
viewer = napari.Viewer()
# viewer.add_labels(gfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
#viewer.add_labels(rfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
#viewer.add_tracks(gfp_data, properties=gfp_properties, graph=gfp_graph, name='GFP Tracks')
#viewer.add_tracks(irfp_tracks)#, properties=rfp_properties, graph=rfp_graph, name='RFP Tracks')
# viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")
    
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')